In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
from scipy.spatial.transform import Rotation as R
import servoing

# use homogenous coordinates 

# ground truth values for transformation
R_gt = (.5, .2, 1.0)
T_gt = (-.4 ,.4, -.5)

#R_gt = (0, 0, 0)
#T_gt = (-.1 , .4, 0)

r = R.from_euler("xyz",R_gt)
r_matrix = np.eye(4)
r_matrix[:3, :3] = r.as_matrix()
r_matrix[:3, 3] = T_gt 

print(r_matrix)

In [ ]:
num_points = 256
points = np.random.random((num_points, 3))-.5
ones = np.ones((num_points, 1))
points = np.concatenate((points, ones), axis=1)
points[:, 2] += 2

points_new = (r_matrix @ points.T).T
mean_z = points[:, 2].mean()

print("mean_z", mean_z)

# camera projection
def C(P, pad=False):
    # input: array 4xn homogenous R^3 coords
    # output: array 4xn homogenous R^2 (but padded for solve_transform)
    proj = P[:, :2] / P[:, 2:3]
    ones = np.ones((len(P), 1))
    if pad:
        res = np.concatenate((proj, ones, ones), axis=1)
    else:
        res = np.concatenate((proj, ones), axis=1)
    return res


# Least-Squares Solution

In [ ]:
from imp import reload
reload(servoing)

# do rigid transformation in R^3 (instead of R^2)
project = True
pad = True
if project:
    P = C(points,pad=pad)
    Q = C(points_new,pad=pad)
else:
    P = points
    Q = points_new


P_initial = P.copy()
Q_initial = Q.copy()

diff_all = []
diff_xy = []
diff_z = []

r_pred = np.eye(4)
servo_steps = 4
for ss in range(servo_steps+2):
    only_log = (ss == servo_steps+1)
    dont_step = (ss == servo_steps)

    # error in R^3
    points_new_pred = (r_pred @ points.T).T
    d_tmp = points_new_pred-points_new
    
    d_all = np.linalg.norm(d_tmp, axis=0).mean()
    d_xy  = np.linalg.norm(d_tmp[:,0:2], axis=0).mean()
    d_z   = np.linalg.norm(d_tmp[:,2:3], axis=0).mean()
    diff_all.append(d_all)
    diff_xy.append(d_xy)
    diff_z.append(d_z)
    print("log")
    
    if only_log:
        # just run the code above to log
        continue
    
    # optimize using R^2 (in R^3)
    # solver only has access to projected values
    guess = servoing.solve_transform(P, Q)
    if guess.shape == (3,3):
        print("XXX")
        #project R^2 to R^3
        tmp = np.eye(4)
        tmp[:3,:3] = guess
        guess = tmp
    
    #guess = solve_transform(points, points_new)
    r_pred = r_pred @ guess
    print("solve")
    
    if dont_step:
        continue
            
    print("step")
    # reproject gt, equivalent to moving robot and re-observing
    Q = (np.linalg.inv(r_pred) @ points_new.T).T
    Q = C(Q, pad=pad)


# I would expect proj error to be a function of depth error
proj_err = np.linalg.norm(P[:,:2]-Q[:,:2], axis=1)

"""
dist_dev = points_new[:,2] - mean_z
dist_err = P[:,0]**2 + P[:,1]**2
#dist_err = np.mean(Q[:, :2] / P[:, :2] - 1,axis=1)
fig,ax = plt.subplots(1)
ax.scatter(dist_err, proj_err, c=dist_dev)
ax.set_xlabel("radial distance")
ax.set_ylabel("projection error")
plt.show()
print(r_pred)
"""

tmp = P[:, :2] / Q[:, :2] - 1
e = tmp
print("points  z", points[:,2].mean())
print("points* z", points_new[:,2].mean())
print("diff z", points_new[:,2].mean() - points[:,2].mean())
print("gt   z", T_gt[2])
print("final loss", d_all)
print(e.mean())
print(r_pred)

fig,ax = plt.subplots(1)
ax.plot(diff_all, '.-',label="all")
ax.plot(diff_xy, '.-',label="xy")
ax.plot(diff_z, '.-', label="z")
ax.set_title("Least-Squares Fitting")
plt.legend()
plt.show()

In [ ]:
# scatter plot
fix, s_ax = plt.subplots(1, 2)
s_ax[0].scatter(P_initial[:, 0], P_initial[:, 1])
s_ax[0].scatter(P[:, 0], P[:, 1])
s_ax[0].set_aspect('equal', 'datalim')
s_ax[0].set_title("P")

Q_hyp = (r_pred @ points.T).T

# scatter plot
s_ax[1].scatter(Q_initial[:, 0], Q_initial[:, 1], label="obs")
s_ax[1].scatter(Q_hyp[:, 0], Q_hyp[:, 1], label="pred")
s_ax[1].set_aspect('equal', 'datalim')
s_ax[1].set_title("Q")
s_ax[1].legend()
plt.show()

In [ ]:
def quat2mat(quat):
    """Convert quaternion coefficients to rotation matrix.
    Args:
        quat: first three coeff of quaternion of rotation. fourth is then computed to have a norm of 1 -- size = [B, 3]
    Returns:
        Rotation matrix corresponding to the quaternion -- size = [B, 3, 3]
    """
    norm_quat = torch.cat([quat[:,:1].detach()*0 + 1, quat], dim=1)
    norm_quat = norm_quat/norm_quat.norm(p=2, dim=1, keepdim=True)
    w, x, y, z = norm_quat[:,0], norm_quat[:,1], norm_quat[:,2], norm_quat[:,3]

    B = quat.size(0)

    w2, x2, y2, z2 = w.pow(2), x.pow(2), y.pow(2), z.pow(2)
    wx, wy, wz = w*x, w*y, w*z
    xy, xz, yz = x*y, x*z, y*z

    rotMat = torch.stack([w2 + x2 - y2 - z2, 2*xy - 2*wz, 2*wy + 2*xz,
                          2*wz + 2*xy, w2 - x2 + y2 - z2, 2*yz - 2*wx,
                          2*xz - 2*wy, 2*wx + 2*yz, w2 - x2 - y2 + z2], dim=1).reshape(B, 3, 3)
    return rotMat


# Hacky Bundle Ajdustment

This optimizes depth only.

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F

lr = 1e-3
optimizer = 'rmsprop'

if optimizer == 'adam':
    Optim = torch.optim.Adam
elif optimizer == 'rmsprop':
    Optim = torch.optim.RMSprop
else:
    Optim = torch.optim.SGD
print('optimizer: ', optimizer)
print('lr: ', lr)

def C_pt(P):
    # input: array 4xn homogenous R^3
    # output array 2xn homogenous R^2
    proj = P[:, :2]/P[:, 2:3]
    #ones = torch.ones((len(P), 1))
    #res = torch.cat((proj, ones), axis=1)
    return proj


P = C_pt(torch.Tensor(points))
Q = C_pt(torch.Tensor(points_new))
f_x = 1
f_y = 1

project_both_ways = True  # lift from P only or Q too
opt_depth = True  # deepth optimized or constant

if opt_depth:
    depth_pred_pt = Variable(torch.FloatTensor(mean_z*torch.ones(num_points)), requires_grad=True)
else:
    depth_pred_pt = torch.ones(num_points)


total_steps = 1000
servo_steps = 0 # 0 for none
r_cuml = torch.eye(4)

R_mse_collect = []
loss_collect = []


for ss in range(servo_steps+1):
    last_servo_step = (ss == servo_steps)
    
    quat_pred_pt = torch.tensor(((0.,0.,0.),), requires_grad=True)
    trns_pred_pt = torch.tensor(((0.,0.,0.),), requires_grad=True)
    if opt_depth:
        proj_optimizer = Optim((quat_pred_pt, trns_pred_pt, depth_pred_pt), lr=lr)
    else:
        proj_optimizer = Optim((quat_pred_pt, trns_pred_pt), lr=lr)
        
    #proj_optimizer_state = #print(proj_optimizer.state_dict())
    
    for i in range(int(total_steps/(servo_steps+1))):
        # compute transformation from variables
        r_pred_pt = torch.eye(4)
        r_pred_pt[:3,:3] = quat2mat(quat_pred_pt)[0]
        r_pred_pt[:3,3] = trns_pred_pt

        # compute error
        with torch.no_grad():
            d = F.mse_loss(torch.Tensor(r_matrix), r_cuml @ r_pred_pt)
            #print(torch.inverse(torch.Tensor(r_matrix)) @ (r_cuml @ r_pred_pt))
            R_mse_collect.append(d.item())

        # take gradient step
        proj_loss = 0
        # this uses projected points P, not points
        x = 1/f_x*P[:, 0]*depth_pred_pt
        y = 1/f_y*P[:, 1]*depth_pred_pt
        z = depth_pred_pt
        o = torch.ones(len(P))
        recon_points = torch.stack((x,y,z,o), axis=1)
        recon_points_f = (r_pred_pt @ recon_points.T).T
        Q_pred = C_pt(recon_points_f)
        proj_loss += F.mse_loss(Q, Q_pred)
        
        #proj_loss += torch.abs(depth_pred_pt.mean() - 2)
        
        if project_both_ways:
            x = 1/f_x*Q[:,0]*depth_pred_pt
            y = 1/f_y*Q[:,1]*depth_pred_pt
            #z = depth_pred_pt  # this was wrong
            z = recon_points_f[:,2]
            o = torch.ones(len(P))
            recon_points_new = torch.stack((x,y,z,o), axis=1)
            recon_points_new_f = (torch.inverse(r_pred_pt) @ recon_points_new.T).T
            P_pred = C_pt(recon_points_new_f)
            proj_loss +=  F.mse_loss(P, P_pred)
            proj_loss +=  F.mse_loss(recon_points_f, recon_points_new)
            #proj_loss +=  F.mse_loss(recon_points, recon_points_new_f)
            
        loss_collect.append(proj_loss.item())
        
        proj_optimizer.zero_grad()
        proj_loss.backward()
        proj_optimizer.step()

    print("end step")
    with torch.no_grad():
        r_cuml = r_cuml @ r_pred_pt
        
        if servo_steps > 0 and not last_servo_step:
            print("step")
            # reproject gt, equivalent to moving robot and re-observing
            Q = C_pt((torch.inverse(r_cuml) @ torch.Tensor(points.T)).T)

print("final loss: ", loss_collect[-1])

print()
print("ground truth: ")
print(r_matrix)
print()
print("prediction: ")
print(r_cuml.detach().numpy().round(3))

print("mean z (gt)  :", mean_z)
print("mean z (pred): ", depth_pred_pt.mean().item())
fig, ax1 = plt.subplots()
ax2 = ax1.twinx() 

l1 = ax1.plot(loss_collect, color="tab:blue", label="loss")
l2 = ax2.plot(R_mse_collect, color="tab:red", label="R_mse")

lns = l1+l2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs,loc='center right')
plt.show()

In [ ]:
# scatter plot
Q_final = C_pt( (r_cuml @ torch.Tensor(points.T)).T).detach().numpy()

fig, s_ax = plt.subplots(1, 2)
s_ax[0].scatter(P_pred[:, 0].detach().numpy(), P_pred[:, 1].detach().numpy(),label="pred")
s_ax[0].scatter(P[:, 0], P[:, 1],label="obs")
s_ax[0].set_aspect('equal', 'datalim')
s_ax[0].set_title("P")
s_ax[0].legend()

# scatter plot
s_ax[1].scatter(Q_initial[:, 0], Q_initial[:, 1], label="obs")
s_ax[1].scatter(Q_final[:, 0], Q_final[:, 1], label="pred")
s_ax[1].set_aspect('equal', 'datalim')
s_ax[1].set_title("Q")
plt.show()

In [ ]:
fig, ax = plt.subplots()
# depth to real
#ax.scatter(points[:,1], depth_pred_pt.tolist())
recon_py = recon_points.detach().numpy()
ax.scatter(points[:,0], recon_py[:,0],label="x")
ax.scatter(points[:,1], recon_py[:,1],label="y")
ax.scatter(points[:,2], depth_pred_pt.tolist(),label="z")
ax.set_xlabel("gt value")
ax.set_ylabel("predicted value")
plt.legend()
plt.show()

fig, ax = plt.subplots()
recon_new_py = recon_points_new.detach().numpy()
ax.scatter(points_new[:,0], recon_new_py[:,0],label="x")
ax.scatter(points_new[:,1], recon_new_py[:,1],label="y")
ax.scatter(points_new[:,2], recon_new_py[:,2],label="z")
ax.set_xlabel("gt value")
ax.set_ylabel("predicted value")
plt.legend()
plt.show()

In [ ]:
num_points


# Classic Bundle Adjustment

This optimizes 3D points.

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F

P = C_pt(torch.Tensor(points))
Q = C_pt(torch.Tensor(points_new))
Q_initial = Q.clone()

inital_depth = mean_z
x = 1/f_x*P[:, 0]*inital_depth
y = 1/f_y*P[:, 1]*inital_depth
z = torch.ones(num_points)*inital_depth
o = torch.ones(num_points)
X_initial = torch.stack((x,y,z,o), axis=1)
X_pred_pt = X_initial.clone().detach().requires_grad_(True)

total_steps = 20000
servo_steps = 0 # 0 for none
r_cuml = torch.eye(4)

R_mse_collect = []
loss_P_collect = []
loss_Q_collect = []
for ss in range(servo_steps+1):
    last_servo_step = (ss == servo_steps)
    
    quat_pred_pt = torch.tensor(((0.,0.,0.),), requires_grad=True)
    trns_pred_pt = torch.tensor(((0.,0.,0.),), requires_grad=True)
    proj_optimizer = Optim((quat_pred_pt, trns_pred_pt, X_pred_pt), lr=lr)
        
    for i in range(int(total_steps/(servo_steps+1))):

        # compute transformation from variables
        r_pred_pt = torch.eye(4)
        r_pred_pt[:3,:3] = quat2mat(quat_pred_pt)[0]
        r_pred_pt[:3,3] = trns_pred_pt

        # compute error
        with torch.no_grad():
            cur_cuml = r_cuml @ r_pred_pt
            #d = F.mse_loss(torch.Tensor(r_matrix), cur_cuml)  # matrix diff
            #d = ((points[:,2].mean()-X_pred_pt[:,2].mean())**2)**.5  # mean_z diff
            d = F.mse_loss(torch.Tensor(points), X_pred_pt)
            R_mse_collect.append(d.item())
            
        if i % 1000 == 0:
            with torch.no_grad():
                Q_final = C_pt((r_pred_pt   @ X_pred_pt.T).T)
                P_pred = C_pt((torch.eye(4) @ X_pred_pt.T).T)

                fig, s_ax = plt.subplots(1, 2)
                s_ax[0].scatter(P_pred[:, 0].detach().numpy(), P_pred[:, 1].detach().numpy(),label="pred")
                s_ax[0].scatter(P[:, 0], P[:, 1],label="obs")
                s_ax[0].set_aspect('equal', 'datalim')
                s_ax[0].set_title("P")
                s_ax[0].legend()

                # scatter plot
                s_ax[1].scatter(Q_final[:, 0], Q_final[:, 1], label="pred")
                s_ax[1].scatter(Q_initial[:, 0], Q_initial[:, 1], label="obs")
                s_ax[1].set_aspect('equal', 'datalim')
                s_ax[1].set_title("Q")
                s_ax[1].legend()
                plt.show()


        # take gradient step
        proj_loss = 0
        
        # this uses projected points P, not points
        P_pred = C_pt((torch.eye(4) @ X_pred_pt.T).T)
        Q_pred = C_pt((r_pred_pt    @ X_pred_pt.T).T)
        loss_P = F.mse_loss(P, P_pred)
        loss_Q = F.mse_loss(Q, Q_pred)
        proj_loss += loss_P + loss_Q #+ F.mse_loss(X_pred_pt[:,3],torch.zeros(num_points))
        
        loss_P_collect.append(loss_P.item())
        loss_Q_collect.append(loss_Q.item())
        
        proj_optimizer.zero_grad()
        proj_loss.backward()
        proj_optimizer.step()
        
        
        
    print("end step")
    with torch.no_grad():
        r_cuml = r_cuml @ r_pred_pt
        
        if servo_steps > 0 and not last_servo_step:
            print("step ")
            # reproject gt, equivalent to moving robot and re-observing
            Q = C_pt((torch.inverse(r_cuml) @ torch.Tensor(points.T)).T)

print("final loss: ", proj_loss.item())

print()
print("ground truth: ")
print(r_matrix)
print()
print("prediction: ")
print(r_cuml.detach().numpy().round(3))

print("mean z (gt)  :", mean_z)
print("mean z (pred): ", depth_pred_pt.mean().item())
fig, ax1 = plt.subplots()
ax1.set_title("Reconstruct Points")
ax2 = ax1.twinx() 

l1 = ax1.plot(loss_P_collect, color="tab:green", label="loss_P")
l2 = ax1.plot(loss_Q_collect, color="tab:blue", label="loss_Q")
l3 = ax2.plot(R_mse_collect, color="tab:red", label="R_mse")

lns = l1+l2+l3
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs,loc='center right')
plt.show()

In [ ]:

# scatter plot
with torch.no_grad():
    Q_final = C_pt((r_cuml @ X_pred_pt.T).T).detach().numpy()

fig, s_ax = plt.subplots(1, 2)
s_ax[0].scatter(P_pred[:, 0].detach().numpy(), P_pred[:, 1].detach().numpy(),label="pred")
s_ax[0].scatter(P[:, 0], P[:, 1],label="obs")
s_ax[0].set_aspect('equal', 'datalim')
s_ax[0].set_title("P")
s_ax[0].legend()

# scatter plot
s_ax[1].scatter(Q_initial[:, 0], Q_initial[:, 1], label="obs")
s_ax[1].scatter(Q_final[:, 0], Q_final[:, 1], label="pred")
s_ax[1].set_aspect('equal', 'datalim')
s_ax[1].set_title("Q")
s_ax[1].legend()
plt.show()

In [ ]:
X_np = X_pred_pt.detach().numpy()
X_new_np = (r_cuml @ X_pred_pt.T).T.detach().numpy()

fig, ax = plt.subplots(1,2)
# depth to real
#ax.scatter(points[:,1], depth_pred_pt.tolist())
recon_py = recon_points.detach().numpy()
ax[0].scatter(points[:,0], X_np[:,0],label="x")
ax[0].scatter(points[:,1], X_np[:,1],label="y")
ax[0].scatter(points[:,2], X_np[:,2],label="z")
ax[0].set_xlabel("gt value")
ax[0].set_ylabel("predicted value")
ax[0].set_title("X vs points")
ax[0].legend()

recon_new_py = recon_points_new.detach().numpy()
ax[1].scatter(points_new[:,0], X_new_np[:,0],label="x")
ax[1].scatter(points_new[:,1], X_new_np[:,1],label="y")
ax[1].scatter(points_new[:,2], X_new_np[:,2],label="z")
ax[1].set_xlabel("gt value")
ax[1].set_ylabel("predicted value")
ax[1].set_title("X* vs points_new")
ax[1].legend()
plt.show()

# Rays



In [ ]:
# do rigid transformation in R^3 (instead of R^2)
pad = True
P = C(points)
Q = C(points_new)

# scatter plot
fix, s_ax = plt.subplots(1, 1)
s_ax.scatter(P[:, 0], P[:, 1])
s_ax.scatter(Q[:, 0], Q[:, 1])
s_ax.set_aspect('equal', 'datalim')
s_ax.set_title("P")
plt.show()

cross = np.cross(P, Q)
norm = np.linalg.norm(cross, axis=1)
M = cross/norm[:,np.newaxis]
u, s, vh = np.linalg.svd(M)

print("eigenvector", s)
print("matrix", vh)
print("shape", vh.shape)
print()

v = vh[2,:]
print(v)
print("pr result", (v/np.linalg.norm(v)).round(4))
print("gt result", (T_gt/np.linalg.norm(T_gt)).round(4))
print()


In [ ]:
import cv2

K = ((1,0,0),
     (0,1,0),
     (0,0,1))

H,status = cv2.findHomography(P[:,:2] ,Q[:,:2])
retval, rotations, translations, normals = cv2.decomposeHomographyMat(H,np.array(K))
print(retval)
print()
print(rotations)
print()
print("trn",np.array(translations))
print()
print(H)